In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import json


c:\AJACENT FILES\GENERATIVE AI\Modules\Module Five\Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


In [3]:
def generate_response(user_input):
    inputs = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")
    outputs = model.generate(
        inputs, 
        max_length=150, 
        num_return_sequences=1, 
        no_repeat_ngram_size=2, 
        temperature=0.7
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [4]:
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
    response = generate_response(user_input)
    print(f"Bot: {response}")


c:\AJACENT FILES\GENERATIVE AI\Modules\Module Five\Project\venv\lib\site-packages\transformers\generation\configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Bot: helloThe U.S. Department of Justice has filed a lawsuit against the company that owns the video game company, Electronic Arts, alleging that the game publisher has violated antitrust laws by allowing its games to be sold on the Internet.

The lawsuit, filed in U, D, and E District Court in Washington, alleges that Electronic Sports, Inc. (ESEA) and its subsidiaries, Activision Blizzard (AAPL), Activision Publishing, Ubisoft, EA, Blizzard Entertainment, Warner Bros. Interactive Entertainment and Warner Brothers Interactive, have violated the antitrust law by selling games on a "virtual marketplace" that allows them to sell their games online. The lawsuit also alleges the games are "unfairly priced" and that EA has "failed to
Goodbye!


In [6]:


# Load the dataset
with open("qa_dataset.json", "r", encoding="utf-8") as file:
    dataset = json.load(file)

formatted_data = ""
for item in dataset:
    formatted_data += f"Question: {item['question']}\nAnswer: {item['answer']}\n\n"

# Save the formatted data
with open("qa_dataset.txt", "w", encoding="utf-8") as file:
    file.write(formatted_data)

print("Dataset formatted and saved as 'qa_dataset.txt'")


Dataset formatted and saved as 'qa_dataset.txt'


In [7]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Load dataset
def load_dataset(file_path, tokenizer, block_size=512):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

# Prepare dataset
train_dataset = load_dataset("qa_dataset.txt", tokenizer)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # GPT-2 does not use masked language modeling
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

# Fine-tune the model
trainer.train()

# Save the model
model.save_pretrained("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")

print("Fine-tuning complete! Model saved in './gpt2-finetuned'")

c:\AJACENT FILES\GENERATIVE AI\Modules\Module Five\Project\venv\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


Fine-tuning complete! Model saved in './gpt2-finetuned'
